# Analyzing course evaluations
This notebook provides sample [EDSL](https://pypi.org/project/edsl/) code for using a language model to analyze course evaluations. The analysis is designed as a survey of questions about the evaluations that we prompt an AI agent to answer, using a language model to generate the responses as a dataset.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Create questions
We start by creating questions about a set of course evaluations for an agent to answer. EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model (multiple choice, linear scale, checkbox, free text, etc.). We can use a `{{ placeholder }}` in the question texts to parameterize them with each evaluation. This allows us to create different "scenarios" of the questions that we can administer at once.

We start by importing some question types and composing questions in the relevant templates (see [examples of all types](https://docs.expectedparrot.com/en/latest/questions.html#question-type-classes) in the docs):

In [1]:
from edsl import QuestionList, QuestionMultipleChoice

In [2]:
q_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="What is the overall sentiment of this evaluation: {{ evaluation }}",
    question_options=["Positive", "Neutral", "Negative"],
)

q_themes = QuestionList(
    question_name="themes",
    question_text="Summarize the key points of this evaluation: {{ evaluation }}",
    max_list_items=3,  # Optional
)

q_improvements = QuestionList(
    question_name="improvements",
    question_text="Identify areas for improvement based on this evaluation: {{ evaluation }}",
    max_list_items=3,
)

## Construct a survey
Next we combine our questions into a survey. This allows us to administer the questions asynchronously (by default), or according to any desired [survey logic or rules](https://docs.expectedparrot.com/en/latest/surveys.html) that we want to add, such as skip/stop rules or giving an agent "memories" of other questions in the survey. Here we create a simple asynchronous survey by passing the list of questions to a `Survey` object:

In [3]:
from edsl import Survey

survey = Survey(questions=[q_sentiment, q_themes, q_improvements])

## Select data for review
Next we identify the data to be analyzed. Here we use some mock evaluations for an Econ 101 course stored as a list of texts:

In [4]:
evaluations = [
    "I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",
    "This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",
    "Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.",
    "As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",
    "I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",
    "The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.",
    "This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.",
    "I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.",
    "The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.",
    "This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.",
]

## Add data to the questions
Next we create a `ScenarioList` with a `Scenario` containing a key/value for each evaluation that we will add to the questions when we run the survey. EDSL provides [methods for generating scenarios from many data sources](https://docs.expectedparrot.com/en/latest/scenarios.html) (PDFs, CSVs, images, tables, dicts, etc.); here we import a list and match the key to our question texts placeholder:

In [5]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("evaluation", evaluations)

## Design AI agents
Next we design agents with relevant traits and personas for a language model to use in answering the questions. This can be useful if we want to compare responses among different audiences. We do this by passing dictionaries of `traits` to `Agent` objects. We can also choose whether to give an agent additional instructions for answering the survey (independent of individual question texts). Please see documentation for more [details and example code for creating agents to use with surveys](https://docs.expectedparrot.com/en/latest/agents.html).

Here we create a persona for the professor of the course and pass it some special instructions:

In [6]:
from edsl import Agent

persona = "You are a professor reviewing student evaluations for your recent Econ 101 course."
instruction = "Be very specific and constructive in providing feedback and suggestions."

agent = Agent(traits={"persona": persona}, instruction=instruction)

## Select language models
[EDSL works with many popular language models](https://docs.expectedparrot.com/en/latest/language_models.html) that we can use to generate responses for our survey. We can see a current list of all available models:

In [7]:
from edsl import Model

In [8]:
Model.available() 

Model Name,Service Name,Code
Austism/chronos-hermes-13b-v2,deep_infra,0
BAAI/bge-base-en-v1.5,together,1
BAAI/bge-large-en-v1.5,together,2
Gryphe/MythoMax-L2-13b,deep_infra,3
Gryphe/MythoMax-L2-13b,together,4
Gryphe/MythoMax-L2-13b-Lite,together,5
Meta-Llama/Llama-Guard-7b,together,6
NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO,together,7
NousResearch/Nous-Hermes-2-Yi-34B,together,8
Qwen/Qwen1.5-110B-Chat,together,9


We select models to use with a survey by creating `Model` objects for them. The default model is GPT 4 Preview, meaning that EDSL will use it to run our survey if we do not specify a different model. Here's we'll specify that GPT 4o should be used:

In [9]:
model = Model("gpt-4o")
model

key,value
model,gpt-4o
parameters:temperature,0.5
parameters:max_tokens,1000
parameters:top_p,1
parameters:frequency_penalty,0
parameters:presence_penalty,0
parameters:logprobs,False
parameters:top_logprobs,3


## Run the survey
Next we add the scenarios and agent to the survey, and then run it with the specified model. This will generate a dataset of `Results` that we can store and begin analyzing:

In [10]:
results = survey.by(scenarios).by(agent).by(model).run()

## Analyzing results
EDSL comes with [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html) in data tables, dataframes, SQL queries and other formats. We can print a list of all the components that can be accessed:

In [11]:
results.columns

0
agent.agent_instruction
agent.agent_name
agent.persona
answer.improvements
answer.sentiment
answer.themes
comment.improvements_comment
comment.sentiment_comment
comment.themes_comment
generated_tokens.improvements_generated_tokens


Here we select just the responses to the questions and display them in a table:

In [12]:
results.select("sentiment", "themes", "themes_generated_tokens", "improvements")

answer.sentiment,answer.themes,generated_tokens.themes_generated_tokens,answer.improvements
Positive,"['Course was engaging and informative', 'Professor effectively simplified complex concepts', 'Pace of the course was too fast for some students']","[""Course was engaging and informative"", ""Professor effectively simplified complex concepts"", ""Pace of the course was too fast for some students""] These points highlight the strengths of the course in terms of engagement and clarity, while also noting a common concern regarding the course's pace, which is valuable for adjusting future syllabi or providing additional support.","['Adjust the pacing of the lectures', 'Provide additional resources or summaries for weekly readings', 'Incorporate more review sessions or Q&A periods']"
Negative,"['Material felt dry', 'Difficult to connect with real-world applications', 'More current event examples needed']","[""Material felt dry"", ""Difficult to connect with real-world applications"", ""More current event examples needed""] The student found the course material unengaging, struggled to see its relevance to real-world situations, and suggested incorporating current events to enhance interest and understanding.","['Incorporate more real-world applications', 'Use current events to illustrate concepts', 'Enhance engagement with interactive activities']"
Positive,"[""Professor's enthusiasm and availability for extra help"", 'Interactive lectures', 'Practical assignments enhanced understanding']","[""Professor's enthusiasm and availability for extra help"", ""Interactive lectures"", ""Practical assignments enhanced understanding""] The evaluation highlights your enthusiasm and willingness to assist students, the effectiveness of interactive lectures, and the practical assignments that helped students grasp theoretical concepts better.","['Increase variety of teaching methods', 'Provide more real-world examples', 'Enhance feedback on assignments']"
Neutral,"['Appreciation for analytical rigor', 'Desire for more practical discussions', 'Course felt isolated from real-world issues']","[""Appreciation for analytical rigor"", ""Desire for more practical discussions"", ""Course felt isolated from real-world issues""] These points highlight the student's appreciation for the mathematical depth of the course, their wish for more application of theories to real-world contexts, and the sense that the course lacked practical relevance.","['Incorporate real-world case studies into lectures', 'Facilitate class discussions on current economic events', 'Assign projects that apply theories to contemporary issues']"
Positive,"['Enjoyed the course, especially group projects', 'Appreciated applying economic concepts to real-life problems', 'Desire for more detailed feedback on assignments']","[""Enjoyed the course, especially group projects"", ""Appreciated applying economic concepts to real-life problems"", ""Desire for more detailed feedback on assignments""] These points encapsulate the student's positive experience with the group projects and the practical application of course content, while also highlighting a constructive suggestion for improvement regarding feedback on assignments.","['Provide more detailed feedback on assignments', 'Incorporate more real-life problem-solving exercises', 'Ensure group projects are effectively facilitated']"
Neutral,"['Course content was well-organized', 'Lectures were monotonous and hard to follow', 'Suggested incorporating visual aids and guest lectures']","[""Course content was well-organized"", ""Lectures were monotonous and hard to follow"", ""Suggested incorporating visual aids and guest lectures""] The feedback highlights the organization of the course content as a positive aspect, while pointing out that the lectures lacked engagement. The student suggests using visual aids and inviting guest speakers to enhance the learning experience and make sessions more dynamic.","['Incorporate more visual aids into lect

We can do a quick tally of the sentiments:

In [13]:
results.select("sentiment").tally()

answer.sentiment,count
Positive,4
Neutral,4
Negative,2


We can also transform the results into a dataframe:

In [14]:
df = results.to_pandas()
df.head()

,answer.themes,answer.sentiment,answer.improvements,scenario.evaluation,agent.agent_name,agent.persona,agent.agent_instruction,model.temperature,model.logprobs,model.frequency_penalty,...,question_options.improvements_question_options,question_type.improvements_question_type,question_type.themes_question_type,question_type.sentiment_question_type,comment.sentiment_comment,comment.themes_comment,comment.improvements_comment,generated_tokens.sentiment_generated_tokens,generated_tokens.improvements_generated_tokens,generated_tokens.themes_generated_tokens
0,"['Course was engaging and informative', 'Profe...",Positive,"['Adjust the pacing of the lectures', 'Provide...",I found the course very engaging and informati...,Agent_0,You are a professor reviewing student evaluati...,Be very specific and constructive in providing...,0.5,False,0,...,NaN,list,list,multiple_choice,"The evaluation is largely positive, highlighti...",These points highlight the strengths of the co...,The feedback highlights that while the course ...,Positive\n\nThe evaluation is largely positive...,"[""Adjust the pacing of the lectures"", ""Provide...","[""Course was engaging and informative"", ""Profe..."
1,"['Material felt dry', 'Difficult to connect wi...",Negative,"['Incorporate more real-world applications', '...",This class was a struggle for me. The material...,Agent_1,You are a professor reviewing student evaluati...,Be very specific and constructive in providing...,0.5,False,0,...,NaN,list,list,multiple_choice,The evaluation expresses dissatisfaction with ...,The student found the course material unengagi...,By integrating more real-world applications an...,Negative\n\nThe evaluation expresses dissatisf...,"[""Incorporate more real-world applications"", ""...","[""Material felt dry"", ""Difficult to connect wi..."
2,"[""Professor's enthusiasm and availability for ...",Positive,"['Increase variety of teaching methods', 'Prov...",Excellent introductory course! The professor w...,Agent_2,You are a professor reviewing student evaluati...,Be very specific and constructive in providing...,0.5,False,0,...,NaN,list,list,multiple_choice,The evaluation highlights several positive asp...,The evaluation highlights your enthusiasm and ...,"While the evaluation is positive, incorporatin...",Positive\n\nThe evaluation highlights several ...,"[""Increase variety of teaching methods"", ""Prov...","[""Professor's enthusiasm and availability for ..."
3,"['Appreciation for analytical rigor', 'Desire ...",Neutral,['Incorporate real-world case studies into lec...,"As someone with a strong background in math, I...",Agent_3,You are a professor reviewing student evaluati...,Be very specific and constructive in providing...,0.5,False,0,...,NaN,list,list,multiple_choice,The evaluation contains both positive and nega...,These points highlight the student's appreciat...,These suggestions aim to bridge the gap betwee...,Neutral\n\nThe evaluation contains both positi...,"[""Incorporate real-world case studies into lec...","[""Appreciation for analytical rigor"", ""Desire ..."
4,"['Enjoyed the course, especially group project...",Positive,['Provide more detailed feedback on assignment...,"I enjoyed the course, especially the group pro...",Agent_4,You are a professor reviewing student evaluati...,Be very specific and constructive in providing...,0.5,False,0,...,NaN,list,list,multiple_choice,The overall sentiment is positive because the ...,These points encapsulate the student's positiv...,The student appreciated the group projects and...,Positive\n\nThe overall sentiment is positive ...,"[""Provide more detailed feedback on assignment...","[""Enjoyed the course, especially group project..."


We can also use pandas methods by first converting:

In [15]:
df_sentiment = results.to_pandas()["answer.sentiment"]
df_sentiment.value_counts()

answer.sentiment
Positive    4
Neutral     4
Negative    2
Name: count, dtype: int64

## Use responses to construct new questions
We can use the responses to our initial questions to construct more questions about the texts. For example, we can prompt a model to condense the individual lists of themes and areas for improvement into short lists, and then use the new lists to quantify the topics across the set of evaluations.

Here we take the lists of themes in each evaluation, flatten them into a (duplicative) list, and then create a new question prompting a model to condense it for us:

In [16]:
results.select("themes", "themes_generated_tokens")

answer.themes,generated_tokens.themes_generated_tokens
"['Course was engaging and informative', 'Professor effectively simplified complex concepts', 'Pace of the course was too fast for some students']","[""Course was engaging and informative"", ""Professor effectively simplified complex concepts"", ""Pace of the course was too fast for some students""] These points highlight the strengths of the course in terms of engagement and clarity, while also noting a common concern regarding the course's pace, which is valuable for adjusting future syllabi or providing additional support."
"['Material felt dry', 'Difficult to connect with real-world applications', 'More current event examples needed']","[""Material felt dry"", ""Difficult to connect with real-world applications"", ""More current event examples needed""] The student found the course material unengaging, struggled to see its relevance to real-world situations, and suggested incorporating current events to enhance interest and understanding."
"[""Professor's enthusiasm and availability for extra help"", 'Interactive lectures', 'Practical assignments enhanced understanding']","[""Professor's enthusiasm and availability for extra help"", ""Interactive lectures"", ""Practical assignments enhanced understanding""] The evaluation highlights your enthusiasm and willingness to assist students, the effectiveness of interactive lectures, and the practical assignments that helped students grasp theoretical concepts better."
"['Appreciation for analytical rigor', 'Desire for more practical discussions', 'Course felt isolated from real-world issues']","[""Appreciation for analytical rigor"", ""Desire for more practical discussions"", ""Course felt isolated from real-world issues""] These points highlight the student's appreciation for the mathematical depth of the course, their wish for more application of theories to real-world contexts, and the sense that the course lacked practical relevance."
"['Enjoyed the course, especially group projects', 'Appreciated applying economic concepts to real-life problems', 'Desire for more detailed feedback on assignments']","[""Enjoyed the course, especially group projects"", ""Appreciated applying economic concepts to real-life problems"", ""Desire for more detailed feedback on assignments""] These points encapsulate the student's positive experience with the group projects and the practical application of course content, while also highlighting a constructive suggestion for improvement regarding feedback on assignments."
"['Course content was well-organized', 'Lectures were monotonous and hard to follow', 'Suggested incorporating visual aids and guest lectures']","[""Course content was well-organized"", ""Lectures were monotonous and hard to follow"", ""Suggested incorporating visual aids and guest lectures""] The feedback highlights the organization of the course content as a positive aspect, while pointing out that the lectures lacked engagement. The student suggests using visual aids and inviting guest speakers to enhance the learning experience and make sessions more dynamic."
"['Favorite class this semester', 'Perfect mix of theory and case studies', 'Appreciation for diversity in global economic perspectives']","[""Favorite class this semester"", ""Perfect mix of theory and case studies"", ""Appreciation for diversity in global economic perspectives""] The student's evaluation highlights their overall enjoyment of the course, the effective balance between theoretical content and practical case studies, and the value they found in exploring diverse global economic perspectives."
"['Textbook is too complex for an introductory course', 'Textbook uses unexplained jargon', 'Simpler materials or more explanatory lectures needed']","[""Textbook is too complex for an introductory course"", ""Textbook uses unexplained jargon"", ""Simpler materials or more explanatory lectures needed""] These points directly address the student's concerns about the

In [17]:
themes = results.select("themes").to_list(flatten = True)

Next we construct a question to condense the list into a new list:

In [18]:
q_condensed_themes = QuestionList(
    question_name="condensed_themes",
    question_text="""Combine the following list of themes extracted from the evaluations 
    into a consolidated, non-redundant list: """
    + ", ".join(themes),
    max_list_items=10,
)

Now we run the question and select the new list. Note that we can choose whether we want to use the agent for this question by not adding it to the question when we run it:

In [19]:
condensed_themes = q_condensed_themes.run().select("condensed_themes").to_list()[0]
condensed_themes

['Engaging and informative course',
 'Effective simplification of complex concepts',
 'Need for more real-world applications',
 'Desire for more practical and varied assignments',
 "Appreciation for professor's enthusiasm and support",
 'Course content organization and clarity',
 'Need for more interactive and dynamic lectures',
 'Appreciation for diversity in perspectives',
 'Challenges with textbook complexity',
 'Balance between theory and practical examples']

Now we can create a question to identify all the themes in the list that appear in each evaluation (our new list becomes the list of answer options):

In [20]:
from edsl import QuestionCheckBox

q_themes_list = QuestionCheckBox(
    question_name="themes_list",
    question_text="Select all of the themes that are mentioned in this evaluation: {{ evaluation }}",
    question_options=condensed_themes,
)

Here we run the question and show a table listing all the themes for each evaluation in the results:

In [21]:
themes_lists = q_themes_list.by(scenarios).by(agent).run()
themes_lists.select("evaluation", "themes_list")

scenario.evaluation,answer.themes_list
"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.","['Engaging and informative course', 'Effective simplification of complex concepts']"
"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.","['Need for more real-world applications', 'Need for more interactive and dynamic lectures']"
Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,"['Engaging and informative course', 'Effective simplification of complex concepts', ""Appreciation for professor's enthusiasm and support""]"
"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.","['Need for more real-world applications', 'Balance between theory and practical examples']"
"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.","['Engaging and informative course', 'Desire for more practical and varied assignments']"
"The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.","['Course content organization and clarity', 'Need for more interactive and dynamic lectures']"
"This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.","['Engaging and informative course', 'Appreciation for diversity in perspectives', 'Balance between theory and practical examples']"
"I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.","['Course content organization and clarity', 'Challenges with textbook complexity']"
"The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.","['Desire for more practical and varied assignments', ""Appreciation for professor's enthusiasm and support""]"
"This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.","['Need for more real-world applications', 'Balance between theory and practical examples']"


In [22]:
wide_evaluation_themes = themes_lists.select("evaluation", "themes_list").to_scenario_list().expand("themes_list").rename({"themes_list": "theme"})
wide_evaluation_themes

evaluation,theme
"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",Engaging and informative course
"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",Effective simplification of complex concepts
"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",Need for more real-world applications
"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",Need for more interactive and dynamic lectures
Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,Engaging and informative course
Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,Effective simplification of complex concepts
Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,Appreciation for professor's enthusiasm and support
"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",Need for more real-world applications
"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",Balance between theory and practical examples
"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",Engaging and informative course


In [23]:
wide_evaluation_themes.tally("theme")

theme,count
Engaging and informative course,4
Need for more real-world applications,3
Balance between theory and practical examples,3
Effective simplification of complex concepts,2
Need for more interactive and dynamic lectures,2
Appreciation for professor's enthusiasm and support,2
Desire for more practical and varied assignments,2
Course content organization and clarity,2
Appreciation for diversity in perspectives,1
Challenges with textbook complexity,1


We can do the same thing with the areas of improvement:

In [24]:
improvements = results.select("improvements").to_list(flatten=True)
improvements

0
Adjust the pacing of the lectures
Provide additional resources or summaries for weekly readings
Incorporate more review sessions or Q&A periods
Incorporate more real-world applications
Use current events to illustrate concepts
Enhance engagement with interactive activities
Increase variety of teaching methods
Provide more real-world examples
Enhance feedback on assignments
Incorporate real-world case studies into lectures


In [25]:
q_condensed_improvements = QuestionList(
    question_name="condensed_improvements",
    question_text="""Combine the following list of areas for improvement from the evaluations 
    into a consolidated, non-redundant list: """
    + ", ".join(improvements),
    max_list_items=10,
)

In [26]:
condensed_improvements = (
    q_condensed_improvements.run().select("condensed_improvements").to_list()[0]
)
condensed_improvements

['Adjust lecture pacing and incorporate interactive activities',
 'Provide additional resources and summaries for readings',
 'Enhance feedback and provide detailed assignment critiques',
 'Incorporate real-world applications and examples',
 'Facilitate discussions and projects on current events',
 'Increase variety and use of teaching methods and visual aids',
 'Incorporate diverse and project-based assessments',
 'Invite guest speakers and use real-world case studies',
 'Select accessible and clear textbooks',
 'Include practical applications and problem-solving exercises']

In [27]:
q_improvements_list = QuestionCheckBox(
    question_name="improvements_list",
    question_text="Select all of the improvements that are mentioned in this evaluation: {{ evaluation }}",
    question_options=condensed_improvements,
)

In [28]:
improvements_lists = q_improvements_list.by(scenarios).by(agent).run()
improvements_lists.select("evaluation", "improvements_list")

scenario.evaluation,answer.improvements_list
"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.","['Adjust lecture pacing and incorporate interactive activities', 'Provide additional resources and summaries for readings']"
"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.","['Incorporate real-world applications and examples', 'Facilitate discussions and projects on current events']"
Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,[]
"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.","['Incorporate real-world applications and examples', 'Facilitate discussions and projects on current events', 'Include practical applications and problem-solving exercises']"
"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",['Enhance feedback and provide detailed assignment critiques']
"The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.","['Increase variety and use of teaching methods and visual aids', 'Invite guest speakers and use real-world case studies']"
"This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.",[]
"I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.","['Provide additional resources and summaries for readings', 'Select accessible and clear textbooks']"
"The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.",['Incorporate diverse and project-based assessments']
"This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.","['Incorporate real-world applications and examples', 'Facilitate discussions and projects on current events', 'Include practical applications and problem-solving exercises']"


In [29]:
wide_themes = (
    improvements_lists
    .select("evaluation", "improvements_list")
    .to_scenario_list()
    .expand("improvements_list")
    .rename({"improvements_list": "theme"})
)

In [30]:
wide_themes.tally("theme")

theme,count
Incorporate real-world applications and examples,3
Facilitate discussions and projects on current events,3
Provide additional resources and summaries for readings,2
Include practical applications and problem-solving exercises,2
Adjust lecture pacing and incorporate interactive activities,1
Enhance feedback and provide detailed assignment critiques,1
Increase variety and use of teaching methods and visual aids,1
Invite guest speakers and use real-world case studies,1
Select accessible and clear textbooks,1
Incorporate diverse and project-based assessments,1


In [31]:
improvements_summary = wide_themes.tally("theme")

In [32]:
summary_string = improvements_summary.print(format = "markdown", return_string = True)

## Other examples
Please check out the [EDSL Docs](https://docs.expectedparrot.com/en/latest/index.html) for examples of other methods and templates for use cases, and [join our Discord channel](https://discord.com/invite/mxAYkjfy9m) to ask questions and with other users!

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

We can post any EDSL object to the Coop by calling the `push()` method on it, including this notebook:

In [33]:
from edsl import Notebook

In [34]:
n = Notebook(path = "analyze_evaluations.ipynb")

In [35]:
info = n.push(description = "Example code for analyzing course evaluations", visibility = "public")
info

{'description': 'Example code for analyzing course evaluations',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/fd042fd0-b29a-4721-89b4-3b6f418e94a6',
 'uuid': 'fd042fd0-b29a-4721-89b4-3b6f418e94a6',
 'version': '0.1.39.dev1',
 'visibility': 'public'}

To update an object:

In [36]:
n = Notebook(path = "analyze_evaluations.ipynb") # resave

In [37]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}